In [1]:
import pip

package_names = ['lightgbm', 'awswrangler', 'sagemaker'] 
pip.main(['install'] + package_names + ['--upgrade'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


0

In [2]:
# Nativos
from dateutil.relativedelta import relativedelta
from time import gmtime, strftime
from datetime import datetime
import random as rn
import joblib
import json
import sys
import os
import gc

#nube
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
import awswrangler as wr
import sagemaker
import boto3

#calculo
import pandas as pd
import numpy as np
import scipy

#grafico
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")

#Interacciones con output
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

gc.collect()
# MODELS
from lightgbm import LGBMClassifier

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)
#import scorecardpy as sc

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

NumExpr defaulting to 4 threads.


### COnfig

In [3]:
proyecto = 'propension'

now = datetime.now()
sess = sagemaker.session.Session()
s3 = boto3.resource('s3')
bucket = sess.default_bucket() 
region = boto3.Session().region_name

print("bucket: ", bucket)
print("region: ", region)
print("now: ", now)

bucket:  sagemaker-us-east-1-058528764918
region:  us-east-1
now:  2021-08-02 18:48:40.970161


### Seteado utilitario

In [4]:
BASE_DIR

'/home/ec2-user/SageMaker'

In [5]:
path_instancia_util = os.path.join(BASE_DIR, 'PROPENSION/UTILITARIO_CODE/utils.py')
path_instancia_util

'/home/ec2-user/SageMaker/PROPENSION/UTILITARIO_CODE/utils.py'

In [6]:
uri_code = 's3://{}/vpc/{}/code'.format(bucket, proyecto)
prefix_utils = '{}/utils.py'.format(uri_code.split(bucket)[-1][1:])
prefix_utils

'vpc/propension/code/utils.py'

In [7]:
s3.Bucket(bucket).upload_file(path_instancia_util, prefix_utils)

### RUtas

In [8]:
uri_output = 's3://{}/vpc/{}/output'.format(bucket, proyecto)
uri_seguimiento = '{}/seguimiento.csv'.format(uri_output)
uri_validation = '{}/validation.csv'.format(uri_output)
uri_copy_seg = '{}/seguimiento.csv'.format(uri_output.replace('output', 'seguimiento'))
uri_copy_val = '{}/validation.csv'.format(uri_output.replace('output', 'seguimiento'))

print('uri_output:', uri_output)
print('uri_seguimiento:', uri_seguimiento)
print('uri_validation:', uri_validation)
print('uri_copy_seg:', uri_copy_seg)
print('uri_copy_val:', uri_copy_val)

uri_raw = 's3://{}/vpc/{}/raw'.format(bucket, proyecto)
print('uri_raw   :', uri_raw)

uri_output: s3://sagemaker-us-east-1-058528764918/vpc/propension/output
uri_seguimiento: s3://sagemaker-us-east-1-058528764918/vpc/propension/output/seguimiento.csv
uri_validation: s3://sagemaker-us-east-1-058528764918/vpc/propension/output/validation.csv
uri_copy_seg: s3://sagemaker-us-east-1-058528764918/vpc/propension/seguimiento/seguimiento.csv
uri_copy_val: s3://sagemaker-us-east-1-058528764918/vpc/propension/seguimiento/validation.csv
uri_raw   : s3://sagemaker-us-east-1-058528764918/vpc/propension/raw


In [9]:
%%time
s3.meta.client.copy({
    'Bucket': bucket,
    'Key': uri_seguimiento.split(bucket)[-1][1:]
},
    bucket,
    uri_copy_seg.split(bucket)[-1][1:]
)

CPU times: user 651 ms, sys: 23.3 ms, total: 674 ms
Wall time: 8.74 s


In [10]:
%%time
s3.meta.client.copy({
    'Bucket': bucket,
    'Key': uri_validation.split(bucket)[-1][1:]
},
    bucket,
    uri_copy_val.split(bucket)[-1][1:]
)

CPU times: user 512 ms, sys: 11.8 ms, total: 524 ms
Wall time: 8.58 s


In [11]:
path_container_input = '/opt/ml/processing/input'
path_container_utils= path_container_input + '/utils'
path_container_output = path_container_input.replace('input', 'output')

print('path_container_input: ', path_container_input)
print('path_container_output: ', path_container_output)
print('path_container_utils: ', path_container_utils)

path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils


### Desacoplado

In [12]:
processing_tags = [{'Key': 'cost-center', 'Value': 'TF2WorkflowProcessing'}]

sklearn_processor_a = SKLearnProcessor(
    framework_version='0.23-1',
    role=get_execution_role(),
    instance_type='ml.m5.4xlarge',
    instance_count=1,
    tags=processing_tags
)

Same images used for training and inference. Defaulting to image scope: inference.
Defaulting to only available Python version: py3


In [13]:
%%time
processing_job_name = "propension-workflow-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))

sklearn_processor_a.run(
    code='preparate_seguimiento.py',  #'/opt/ml/processing/codigo_proceso/' , '/opt/ml/processing/input/code'
    job_name=processing_job_name,
    inputs=[
        ProcessingInput(
            source=uri_output.replace('output', 'seguimiento'),
            destination=path_container_input,
            s3_data_distribution_type='ShardedByS3Key'
        ),
        ProcessingInput(
            source=uri_code,
            destination=path_container_utils,
            s3_data_distribution_type='ShardedByS3Key'
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name='result',
            destination=uri_output.replace('output', 'seguimiento'),
            source=path_container_output
        )
    ]
)

preprocessing_job_description = sklearn_processor_a.jobs[-1].describe()


Job Name:  propension-workflow-2021-08-02-18-48-58
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/propension/seguimiento', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/propension/code', 'LocalPath': '/opt/ml/processing/input/utils', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/propension-workflow-2021-08-02-18-48-58/input/code/preparate_seguimiento.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType

In [14]:
a = None

In [15]:
a is not None

False